# Assignment 2: Language Modelling in Hangman

Student Name: Junjie Xia

Student ID: 1045673

## General info

<b>Due date</b>: Friday, 8 April 2022 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python page" on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [2]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [3]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [4]:
interactive = False
#interactive = True

<b>For your testing:</b>

You can play the game interactively using the following command:

In [5]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [6]:
from nltk.corpus import brown
import numpy as np

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###
brown_words = brown.words()
brown_words = [token.lower() for token in brown_words]
middle_list = []
#print(type(brown_words))
for i in brown_words:
    if i.encode('utf-8').isalpha() and i not in middle_list:
        middle_list.append(i)
#print(middle_list[:2000])
np.random.shuffle(middle_list)
test_set = middle_list[:1000]
training_set = middle_list[1000:]

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [7]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [8]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [9]:
def test_guesser(guesser, test):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [10]:
import random

def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
    while True:
        randlowercase = chr(random.randint(ord('a'), ord('z')))
        if randlowercase not in guessed:
            break
    return randlowercase.lower().strip()
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = gadgets
Number of mistakes made by the random guesser = 19

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.673


<b>For your testing:</b>

In [11]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [12]:
unigram_counts = None

###
# Your answer BEGINS HERE
###
unigram_counts = {}
for i in training_set:
    for j in i:
        if j not in unigram_counts:
            unigram_counts[j] = 1
        else:
            number = unigram_counts[j]
            number = number + 1
            unigram_counts[j] = number
#print(unigram_counts_dict)
#unigram_counts = sorted(unigram_counts.keys(),reverse = True)
unigram_counts = {k: v for k, v in sorted(unigram_counts.items(), key=lambda item: item[1], reverse=True)}
#print(unigram_counts)
###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    while True:
        for i in list(unigram_counts):
            if i not in guessed:
                #print('mask:',mask)
                return i
    #return
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.344


<b>For your testing:</b>

In [13]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [14]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###
unigram_counts_by_length = {}
for i in training_set:
    if len(i) in unigram_counts_by_length:
        unigram_counts_by_length_middle = unigram_counts_by_length[len(i)]
        for j in i:
            if j not in unigram_counts_by_length_middle:
                unigram_counts_by_length_middle[j] = 1
            else:
                number = unigram_counts_by_length_middle[j]
                number = number + 1
                unigram_counts_by_length_middle[j] = number
        unigram_counts_by_length_middle = {k: v for k, v in sorted(unigram_counts_by_length_middle.items(), key=lambda item: item[1], reverse=True)}
        unigram_counts_by_length[len(i)] = unigram_counts_by_length_middle
        #print('unigram_counts_by_length:(1)',unigram_counts_by_length)
    else:
        unigram_counts_by_length_middle = {}
        for j in i:
            if j not in unigram_counts_by_length_middle:
                unigram_counts_by_length_middle[j] = 1
                #print('unigram_counts_by_length_middle',unigram_counts_by_length_middle)
            else:
                number = unigram_counts_by_length_middle[j]
                number = number + 1
                unigram_counts_by_length_middle[j] = number
        unigram_counts_by_length_middle = {k: v for k, v in sorted(unigram_counts_by_length_middle.items(), key=lambda item: item[1], reverse=True)}
        unigram_counts_by_length[len(i)] = unigram_counts_by_length_middle
        #print('unigram_counts_by_length', unigram_counts_by_length)
#print(unigram_counts_by_length)

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    if len(mask) in unigram_counts_by_length:
        possible_dict = unigram_counts_by_length[len(mask)]
        while True:
            for i in list(possible_dict):
                if i not in guessed:
                    return i
            for i in list(unigram_counts):
                if i not in guessed:
                    return i
    else:
        while True:
            for i in list(unigram_counts):
                if i not in guessed:
                    return i

    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.313


<b>For your testing:</b>

In [15]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Note**:
- When backing-off to the unigram language model, you **must use the vanilla unigram language model that you have built in Q3**. Do not use the length-based unigram language model, or any fancy variant, or you will lose marks.
- You should build a **standard bigram language model**; i.e. do not do anything complicated like a bidirectional bigram language model.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [19]:
bigram_counts = None

###
# Your answer BEGINS HERE
###
#reference:https://stackoverflow.com/questions/12488722/counting-bigrams-pair-of-two-words-in-a-file-using-python
from collections import Counter
from nltk.util import ngrams 

#text = "the quick person did not realize his speed and the quick person bumped "
#n_gram = 2
#bigram_counts = Counter(ngrams(training_set, n_gram))
#print(a)
bigram_counts_middle = []
for i in training_set:
    bigram_counts_middle = bigram_counts_middle + ['$'] + list(i)
#print(bigram_counts_middle[:100])
n_gram = 2
bigram_counts = Counter(ngrams(bigram_counts_middle, n_gram))
#print(bigram_counts[('i', 'n')])
#print(bigram_counts)

###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_counts=unigram_counts): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    potential_dictionary = {}
    if mask[0] == '_':
        best = 0
        for i in range(ord('a'), ord('z') + 1):
            if chr(i) not in guessed and bigram_counts[('$', chr(i))]/len(training_set) > best:
                best = bigram_counts[('$', chr(i))]/len(training_set)
                letter = chr(i)
#                 print('letter:first',letter)
        if letter in potential_dictionary:
#                 number = potential_dictionary[letter]
#                 potential_dictionary[letter] = number + best
            if potential_dictionary[letter] < best:
                potential_dictionary[letter] = best
        else:
            potential_dictionary[letter] = best
    for i in range(len(mask)-1):
        if mask[i+1] == len(mask):
            best = 0
            letter = ''
            for j in range(ord('a'), ord('z') + 1):
                if chr(j) not in guessed and bigram_counts[(chr(j), '$')]/unigram_counts[chr(j)] > best:
                    best = bigram_counts[(chr(j), '$')]/unigram_counts[chr(j)]
                    letter = chr(j)
            if letter in potential_dictionary:
#                 number = potential_dictionary[letter]
#                 potential_dictionary[letter] = number + best
                if potential_dictionary[letter] < best:
                    potential_dictionary[letter] = best
            else:
                potential_dictionary[letter] = best
        if mask[i] != '_' and mask[i+1] == '_':
            best = 0
            letter = ''
            for j in range(ord('a'), ord('z') + 1):
                if chr(j) not in guessed and bigram_counts[(mask[i], chr(j))]/unigram_counts[mask[i]] > best:
                    best = bigram_counts[(mask[i], chr(j))]/unigram_counts[mask[i]]
                    letter = chr(j)
#                     print('letter:',letter)
            if letter in potential_dictionary:
#                 number = potential_dictionary[letter]
#                 potential_dictionary[letter] = number + best
                if potential_dictionary[letter] < best:
                    potential_dictionary[letter] = best
            else:
                potential_dictionary[letter] = best
    #for i in range(ord('a'), ord('z')+1):
        #bigram_counts[(chr(i))]
    potential_dictionary = {k: v for k, v in sorted(potential_dictionary.items(), key=lambda item: item[1], reverse=True)}
#     print('potential_dictionary:(out)', list(potential_dictionary)[0])
#     print('potential_dictionary:(out)', list(potential_dictionary))
#     if list(potential_dictionary)[0] == '':
#         print('mask:',mask)
#     if '$' in potential_dictionary:
#         print('it has $')
    if list(potential_dictionary)[0] == '':
        while True:
            for i in list(unigram_counts):
                if i not in guessed:
                    return i
    return list(potential_dictionary)[0]

    ###
    # Your answer ENDS HERE
    ###



result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.933


### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 7.6
* 1.0 mark if 7.6 <= x < 8.0
* 0.5 mark if 8.0 <= x < 8.5
* 0.0 mark if x >= 8.5

Note: when testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [18]:
###
# Your answer BEGINS HERE
###

from nltk.lm.preprocessing import flatten
from nltk.util import pad_sequence
from nltk.lm.preprocessing import pad_both_ends
from collections import Counter
from nltk.util import ngrams
from nltk.util import skipgrams

processed_list = list(flatten(pad_both_ends(sent, n=2) for sent in training_set))
unigram_counts_update = Counter(processed_list)
bigram_counts_update = Counter(ngrams(processed_list, 2))
n_gram = 3
trigram_counts_update = Counter(ngrams(processed_list, n_gram))
skip_grams_3_words = list(skipgrams(processed_list, 3, 2))
skip_grams_2_words = Counter(skipgrams(processed_list, 2, 2))

def calculate_unigram(item_count, corpus):
    return item_count/len(corpus)

# item is the name
def calculate_bigram(bigram_counts, unigram_counts, item1, item2):
    return bigram_counts[(item1, item2)]/unigram_counts[item1]

def calculate_trigram(trigram_counts, bigram_counts, item1, item2, item3):
    return trigram_counts[(item1, item2, item3)]/bigram_counts[(item1, item2)]

def calculate_skipgram(skip_grams_2_words, unigram_counts, item1, item2):
    return skip_grams_2_words[(item1, item2)]/unigram_counts[item1]

# the function used for reversing
def calculate_rev_unigram(item_count, corpus):
    return item_count/len(corpus)

def calculate_rev_bigram(bigram_counts, unigram_counts, item1, item2):
    return bigram_counts[(item1, item2)]/unigram_counts[item2]

def calculate_rev_trigram(trigram_counts, bigram_counts, item1, item2, item3):
    return trigram_counts[(item1, item2, item3)]/bigram_counts[(item2, item3)]

def calculate_rev_skipgram(skip_grams_2_words, unigram_counts, item1, item2):
    return skip_grams_2_words[(item1, item2)]/unigram_counts[item2]

def my_amazing_ai_guesser(mask, guessed):
    potential_dictionary = {}
    mask_processed = list(pad_sequence(mask,pad_left=True,left_pad_symbol="<s>",pad_right=True,right_pad_symbol="</s>",n=2))
    for i in range(ord('a'), ord('z')+1):
        possibility = 0
        if chr(i) in guessed:
            continue
        for j in range(len(mask_processed)):
            if 1 < j and mask_processed[j] == '_' and mask_processed[j-1] != '_' and mask_processed[j-2] != '_':
                possibility = possibility + calculate_trigram(trigram_counts_update, bigram_counts_update, mask_processed[j-2], mask_processed[j-1], chr(i))
            elif 0 < j and mask_processed[j] == '_' and mask_processed[j-1] != '_':
                possibility = possibility + calculate_bigram(bigram_counts_update, unigram_counts_update, mask_processed[j-1], chr(i))
            elif 1 < j and mask_processed[j] == '_' and mask_processed[j-1] == '_' and mask_processed[j-2] != '_':
                possibility = possibility + calculate_skipgram(skip_grams_2_words, unigram_counts_update, mask_processed[j-2], chr(i))
            elif mask_processed[j] == '_':
                possibility = possibility + calculate_unigram(unigram_counts_update[chr(i)], processed_list)
#             Bidirectional
            if 1 < j and mask_processed[-j-1] == '_' and mask_processed[-j] != '_' and mask_processed[-j+1] != '_':
                possibility = possibility + calculate_rev_trigram(trigram_counts_update, bigram_counts_update, chr(i), mask_processed[-j], mask_processed[-j+1])
            elif 0 < j and mask_processed[-j-1] == '_' and mask_processed[-j] != '_':
                possibility = possibility + calculate_rev_bigram(bigram_counts_update, unigram_counts_update, chr(i), mask_processed[-j])
            elif 1 < j and mask_processed[-j-1] == '_' and mask_processed[-j] == '_' and mask_processed[-j+1] != '_':
                possibility = possibility + calculate_rev_skipgram(skip_grams_2_words, unigram_counts_update, chr(i), mask_processed[-j+1])
            elif mask_processed[j] == '_':
                possibility = possibility + calculate_rev_unigram(unigram_counts_update[chr(i)], processed_list)
        potential_dictionary[chr(i)] = possibility
        potential_dictionary = {k: v for k, v in sorted(potential_dictionary.items(), key=lambda item: item[1], reverse=True)}
    return list(potential_dictionary)[0]
#      return random_guesser(mask, guessed)

    
###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.783


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE

This model used several methods which are introduced in both classes and assignment. Firstly, this model used trigram, bigram and unigram language model, which could used to calculate the possibility of a letter appeared next. Secondly, I used the backoff method to integrating those functions and multiple NLTK packages to improve the data processing speed. Moreover, it also used the bidirectional models to improve the performance and the result improve significantly. In terms of the smoothing function, I attempted to use it, but it decreased the accuracy of the model. Hence, I decided to discard it.

##### Your answer ENDS HERE